In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import scale, StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn import neighbors
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

In [3]:
from warnings import filterwarnings
filterwarnings("ignore")

In [22]:
df = pd.read_csv("reading_data/Hitters.csv")
df = df.dropna()
df = pd.get_dummies(df,["League","Division","NewLeague"],["League_dum","Division_dum","NewLeague_dum"])

In [23]:
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,...,PutOuts,Assists,Errors,Salary,LeagueLeague_dumA,LeagueLeague_dumN,DivisionDivision_dumE,DivisionDivision_dumW,NewLeagueNewLeague_dumA,NewLeagueNewLeague_dumN
1,315,81,7,24,38,39,14,3449,835,69,...,632,43,10,475.0,0,1,0,1,0,1
2,479,130,18,66,72,76,3,1624,457,63,...,880,82,14,480.0,1,0,0,1,1,0
3,496,141,20,65,78,37,11,5628,1575,225,...,200,11,3,500.0,0,1,1,0,0,1
4,321,87,10,39,42,30,2,396,101,12,...,805,40,4,91.5,0,1,1,0,0,1
5,594,169,4,74,51,35,11,4408,1133,19,...,282,421,25,750.0,1,0,0,1,1,0


In [24]:
X = df.drop(["Salary"],axis=1)
y = df["Salary"]
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.2)

## KNN MODEL

In [71]:
model = KNeighborsRegressor().fit(X_train,y_train)
y_pred = model.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

323.40372790119386

In [72]:
r2_score(y_test,y_pred)

0.41316349312098344

In [10]:
RMSE =[]
for k in range(0,10):
    k = k+1
    model = KNeighborsRegressor(n_neighbors=k).fit(X_train,y_train)
    y_pred = model.predict(X_test)
    RMSE.append(np.sqrt(mean_squared_error(y_test,y_pred)))
    print("k=",k,"için RMSE değeri=",np.sqrt(mean_squared_error(y_test,y_pred)) )

k= 1 için RMSE değeri= 302.2948937463125
k= 2 için RMSE değeri= 285.04416783175236
k= 3 için RMSE değeri= 274.90375232039355
k= 4 için RMSE değeri= 282.1949785402674
k= 5 için RMSE değeri= 284.0164295736043
k= 6 için RMSE değeri= 303.8404900529691
k= 7 için RMSE değeri= 302.669882593793
k= 8 için RMSE değeri= 285.5884722120224
k= 9 için RMSE değeri= 295.020526376955
k= 10 için RMSE değeri= 288.3110513778556


## GridSearchCV

In [11]:
knn_params = {"n_neighbors": np.arange(1,30,1)}
knn_model = KNeighborsRegressor()
knn_cv = GridSearchCV(knn_model,knn_params , cv =10).fit(X_train,y_train)

In [12]:
knn_cv.best_params_

{'n_neighbors': 3}

In [13]:
knn_model = KNeighborsRegressor(n_neighbors=knn_cv.best_params_["n_neighbors"]).fit(X_train,y_train)
y_pred = knn_model.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

274.90375232039355

In [14]:
r2_score(y_test,y_pred)

0.595619073111869

## SVM

In [76]:
svr_model = SVR("linear")
svr_model.fit(X_train,y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [79]:
y_pred = svr_model.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

338.4515549536238

In [80]:
r2_score(y_test,y_pred)

-0.13331939089699896

In [19]:
svr_params = {"C": [1,0.5,0.1,0.01,0.001,0.0005]}
svr_cv = GridSearchCV(svr_model,svr_params,cv =10, verbose=2,n_jobs=-1).fit(X_train,y_train)

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  2.4min finished


In [20]:
svr_cv.best_params_

{'C': 0.001}

In [21]:
svr_model = SVR("linear",C=0.001)
svr_model.fit(X_train,y_train)

SVR(C=0.001, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [22]:
y_pred = svr_model.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

329.0480592749908

In [23]:
r2_score(y_test,y_pred)

0.43127559311335306

## YAPAY SİNİR AĞLARI

In [84]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
scaler.fit(X_test)
X_test_scaled = scaler.transform(X_test)

In [85]:
NN_model = MLPRegressor()
NN_model.fit(X_train_scaled,y_train)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=200,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=None, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [86]:
y_pred = NN_model.predict(X_test_scaled)
np.sqrt(mean_squared_error(y_test,y_pred))

637.6361161197352

In [87]:
r2_score(y_test,y_pred)

-0.9416450018964124

In [31]:
NN_params = {"alpha":[1,0.1,0.01,0.001],"hidden_layer_sizes":[(10,10),(30,30),(100,100)]}
NN_cv = GridSearchCV(NN_model,NN_params,cv=10,verbose=2,n_jobs=-1).fit(X_train_scaled,y_train)

Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   11.0s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:   32.1s finished


In [32]:
NN_cv.best_params_

{'alpha': 0.1, 'hidden_layer_sizes': (100, 100)}

In [96]:
NN_model = MLPRegressor(alpha=0.1,hidden_layer_sizes=(100, 100))
NN_model.fit(X_train_scaled,y_train)
y_pred = NN_model.predict(X_test_scaled)
np.sqrt(mean_squared_error(y_test,y_pred))

391.3913204596654

In [97]:
r2_score(y_test,y_pred)

0.024969204353445074

## CART

In [101]:
# Classification and Regression Tree
cart_model = DecisionTreeRegressor().fit(X_train,y_train)

In [102]:
y_pred = cart_model.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

395.27910897837216

In [103]:
r2_score(y_test,y_pred)

0.3374080895144158

In [23]:
cart_params = {"max_depth":np.arange(1,51),"min_samples_split":np.arange(1,41)}
cart_cv = GridSearchCV(cart_model,cart_params,cv=10,verbose =2, n_jobs=-1).fit(X_train,y_train)

Fitting 10 folds for each of 2000 candidates, totalling 20000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 2072 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done 5320 tasks      | elapsed:   16.8s
[Parallel(n_jobs=-1)]: Done 9848 tasks      | elapsed:   31.4s
[Parallel(n_jobs=-1)]: Done 15688 tasks      | elapsed:   50.3s
[Parallel(n_jobs=-1)]: Done 20000 out of 20000 | elapsed:  1.1min finished


In [55]:
cart_cv.best_params_

{'max_depth': 7, 'min_samples_split': 21}

In [46]:
cart_model = DecisionTreeRegressor(max_depth=10,min_samples_split=11).fit(X_train,y_train)
y_pred = cart_model.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

273.8791108411931

In [50]:
r2_score(y_test,y_pred)

0.512031863889143

## Random Forests

In [107]:
random_model = RandomForestRegressor().fit(X_train,y_train)
y_pred = random_model.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

389.575685852801

In [108]:
r2_score(y_test,y_pred)

0.5045241698781977

In [61]:
random_params={"max_depth":[3,5,10,15,20],"min_samples_split":[2,3,5,8,10,15,20,30],"n_estimators":[10,30,50,100,150,300,500,750,1000]}
random_cv = GridSearchCV(random_model,random_params,cv=10,n_jobs=-1,verbose=2).fit(X_train,y_train)

Fitting 10 folds for each of 360 candidates, totalling 3600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   50.6s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed: 13.1min
[Parallel(n_jobs=-1)]: Done 2584 tasks      | elapsed: 17.2min
[Parallel(n_jobs=-1)]: Done 3273 tasks      | elapsed: 22.3min
[Parallel(n_jobs=-1)]: Done 3600 out of 3600 | elapsed: 24.7min finished


In [63]:
random_cv.best_params_

{'max_depth': 15, 'min_samples_split': 30, 'n_estimators': 50}

In [112]:
random_model = RandomForestRegressor(max_depth=15,min_samples_split=30,n_estimators=50).fit(X_train,y_train)
y_pred = random_model.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

205.0470888226582

In [113]:
r2_score(y_test,y_pred)

0.547313913240786

## Değişken Önem Düzeyi

In [137]:
importance= pd.DataFrame({"importance":random_model.fit(X_train,y_train).feature_importances_*100},index=X_train.columns)

In [153]:
importance.sort_values(by="importance",axis=0,ascending=False)

,importance
CRBI,24.129728
CHits,21.510907
Walks,15.423745
CRuns,8.727567
PutOuts,8.180929
CAtBat,4.208657
Hits,3.838415
RBI,3.553294
AtBat,2.919724
Runs,2.221492


In [151]:
importance

,importance
AtBat,2.919724
Hits,3.838415
HmRun,2.053179
Runs,2.221492
RBI,3.553294
Walks,15.423745
Years,0.233203
CAtBat,4.208657
CHits,21.510907
CHmRun,1.555300


## Gradient Boosting Machines

In [24]:
gbm_model = GradientBoostingRegressor().fit(X_train,y_train)
y_pred= gbm_model.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

248.72619224162963

In [25]:
r2_score(y_test,y_pred)

0.5190005526621599

In [158]:
gbm_model

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [38]:
gbm_params={"alpha":[10,0.1,0.01,0.001],"learning_rate":[1,0.1,0.01,0.001],"min_samples_leaf":[1,3,5],"min_samples_split":[2,4,5,10],"n_estimators":[100,200,500]}
gbm_cv = GridSearchCV(gbm_model,gbm_params,cv=10,verbose=2,n_jobs=-1).fit(X_train,y_train)

Fitting 10 folds for each of 576 candidates, totalling 5760 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 1858 tasks      | elapsed:   48.8s
[Parallel(n_jobs=-1)]: Done 2061 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 2344 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 2709 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 3154 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 3681 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 4288 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 4977 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done 5746 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done 5760 out of 5760 | elapsed:  9.3min finished


In [39]:
gbm_cv.best_params_

{'alpha': 0.01,
 'learning_rate': 0.01,
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 500}

In [40]:
gbm_model = GradientBoostingRegressor(alpha= 0.01,
 learning_rate= 0.01,
 min_samples_leaf= 1,
 min_samples_split= 5,
 n_estimators= 500).fit(X_train,y_train)
y_pred= gbm_model.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

230.4473650456836

In [41]:
r2_score(y_test,y_pred)

0.5870998617729569

## XGBoost

In [6]:
!pip install xgboost

In [64]:
import xgboost
from xgboost import XGBRegressor

In [9]:
xgb_model = XGBRegressor()
xgb_model.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
             validate_parameters=False, verbosity=None)

In [10]:
y_pred = xgb_model.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

214.87239696890484

In [11]:
r2_score(y_test,y_pred)

0.569505226963309

In [13]:
xgb_params = {"learning_rate":[0.1,0.01,0.001],"max_depth":[4,6,10],"n_estimators":[200,300,500]}
xbm_cv = GridSearchCV(xgb_model,xgb_params,cv=10,verbose=2,n_jobs=-1).fit(X_test,y_test)

Fitting 10 folds for each of 27 candidates, totalling 270 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   15.4s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   23.1s
[Parallel(n_jobs=-1)]: Done 270 out of 270 | elapsed:   29.6s finished


In [14]:
xbm_cv.best_params_

{'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 300}

In [60]:
xgb_model = XGBRegressor(learning_rate= 0.01, max_depth= 4, n_estimators= 300)
xgb_model.fit(X_train,y_train)
y_pred = xgb_model.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

285.77704881268653

In [61]:
r2_score(y_test,y_pred)

0.3075237254189366

In [43]:
importances= pd.DataFrame({"importances":xgb_model.feature_importances_},index=X_train.columns)

In [44]:
importances.sort_values(by="importances" ,axis=0,ascending=False)

,importances
CHits,0.368883
CRBI,0.120585
RBI,0.061267
CRuns,0.060538
AtBat,0.053678
CHmRun,0.042392
PutOuts,0.039428
Runs,0.038381
Hits,0.036763
CWalks,0.031082


## LightGBM

In [45]:
!pip install lightgbm

In [63]:
from lightgbm import LGBMRegressor

In [16]:
Lgbm_model = LGBMRegressor().fit(X_train,y_train)
y_pred = Lgbm_model.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

253.08626763882629

In [17]:
r2_score(y_test,y_pred)

0.5941582156226433

In [12]:
lgbm_params= {"learning_rate":[0.1,0.01,0.5,1],"max_depth":[-1,2,4,8,10],"n_estimators":[20,50,100,200]}
lgbm_cv = GridSearchCV(Lgbm_model,lgbm_params,cv=10,verbose=2,n_jobs=-1).fit(X_train,y_train)

Fitting 10 folds for each of 80 candidates, totalling 800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 444 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:   20.0s finished


In [13]:
lgbm_cv.best_params_

{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100}

In [18]:
Lgbm_model = LGBMRegressor(learning_rate=0.1,max_depth=4,n_estimators=100).fit(X_train,y_train)
y_pred = Lgbm_model.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

243.6271056844598

In [19]:
r2_score(y_test,y_pred)

0.6239281659973506

## Catboost

In [20]:
!pip install catboost

  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11435 sha256=e5de614e9b199df7a3c7177dacbe64cb868d9dc36f453d0937a9523fb77fa7d1
  Stored in directory: c:\users\darak\appdata\local\pip\cache\wheels\f9\8d\8d\f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying


In [21]:
from catboost import CatBoostRegressor

In [26]:
cat_model = CatBoostRegressor().fit(X_train,y_train)

Learning rate set to 0.029563
0:	learn: 462.2997658	total: 4.64ms	remaining: 4.63s
1:	learn: 456.5191997	total: 10.4ms	remaining: 5.18s
2:	learn: 451.1425015	total: 17.9ms	remaining: 5.95s
3:	learn: 445.2805893	total: 26.4ms	remaining: 6.58s
4:	learn: 439.5203150	total: 33.1ms	remaining: 6.59s
5:	learn: 433.7878754	total: 38.3ms	remaining: 6.35s
6:	learn: 428.4114801	total: 42.9ms	remaining: 6.08s
7:	learn: 422.8563797	total: 47.9ms	remaining: 5.93s
8:	learn: 416.9600899	total: 52.8ms	remaining: 5.81s
9:	learn: 411.9182042	total: 57.6ms	remaining: 5.7s
10:	learn: 406.6327110	total: 62.4ms	remaining: 5.61s
11:	learn: 402.3496983	total: 67.3ms	remaining: 5.54s
12:	learn: 397.8959005	total: 71.8ms	remaining: 5.45s
13:	learn: 393.4074897	total: 76.5ms	remaining: 5.38s
14:	learn: 388.7795034	total: 81.5ms	remaining: 5.35s
15:	learn: 384.1011247	total: 88.3ms	remaining: 5.43s
16:	learn: 379.6564107	total: 96.9ms	remaining: 5.6s
17:	learn: 376.0465673	total: 103ms	remaining: 5.61s
18:	learn: 

In [27]:
y_pred = cat_model.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

220.73127122519873

## Automatization

In [52]:
def Function(Modelx):
    model = Modelx().fit(X_train,y_train)
    y_pred = model.predict(X_test)
    return(np.sqrt(mean_squared_error(y_test,y_pred)))
    

In [31]:
Function(GradientBoostingRegressor())

236.18262254984387

In [32]:
Function(RandomForestRegressor())

253.3970763572999

In [65]:
list=[GradientBoostingRegressor, RandomForestRegressor, KNeighborsRegressor, MLPRegressor, LinearRegression, SVR, DecisionTreeRegressor, LGBMRegressor, XGBRegressor]

In [67]:
def Function(modelx):
    model = modelx().fit(X_train,y_train)
    y_pred = model.predict(X_test)
    print(modelx.__name__,":",np.sqrt(mean_squared_error(y_test,y_pred)))

In [68]:
for i in list:
    Function(i)

GradientBoostingRegressor : 232.41992893872674
RandomForestRegressor : 247.57958521588344
KNeighborsRegressor : 242.67275994924867
MLPRegressor : 258.4082851039541
LinearRegression : 298.0630773563609
SVR : 346.5065901548966
DecisionTreeRegressor : 382.62532408803224
LGBMRegressor : 251.7558450877996
XGBRegressor : 238.3154190060623


In [56]:
model = GradientBoostingRegressor().fit(X_train,y_train)
y_pred = model.predict(X_test)
print(":",np.sqrt(mean_squared_error(y_test,y_pred)))

: 235.2532969394213
